In [ ]:
import pandas as pd
from google.oauth2 import service_account
import pandas_gbq as gbq

In [ ]:
credentials_path = 'token.json'

# Authenticate with your credentials
credentials = service_account.Credentials.from_service_account_file(
    credentials_path, scopes=['https://www.googleapis.com/auth/bigquery'])

# Set the credentials for pandas_gbq
gbq.context.credentials = credentials

project_id = 'capstone-398012'
dataset_id = 'capstone'
table_id = "filled_table_2"

from pandas_gbq import read_gbq

query = f"""
SELECT *
FROM `{project_id}.{dataset_id}.{table_id}`

"""

# Authenticate and read data from BigQuery into a DataFrame
cri_df = read_gbq(query, project_id=project_id, dialect='standard')
cri_df.head()

In [ ]:
file_path3 = 'Datasets/CompMapping.csv'

# Read the CSV file into a pandas DataFrame
mapping_df = pd.read_csv(file_path3)

mapping_df.head()

In [ ]:
mapping_df['Ticker'] = mapping_df.apply(lambda row: row['Ticker'].replace(' US', '') if row['Country_name'] == 'United States' else row['Ticker'], axis=1)
mapping_df['Ticker'] = mapping_df.apply(lambda row: row['Ticker'].replace(' CN', '') if row['Country_name'] == 'Canada' else row['Ticker'], axis=1)

mapping_df = mapping_df.drop(mapping_df[mapping_df['Country_name'] == 'China'].index)

mapping_df['U3_COMPANY_NUMBER'] = mapping_df['U3_COMPANY_NUMBER'].astype(int)

mapping_df

In [ ]:
ticker_df = pd.merge(cri_df, mapping_df[['U3_COMPANY_NUMBER', 'Ticker']],  left_on='CompanyNumber', right_on='U3_COMPANY_NUMBER', how='left')
ticker_df.drop('U3_COMPANY_NUMBER', axis=1, inplace=True)

ticker_df

# Download as csv
# ticker_df.to_csv('ticker_df.csv', index=False)

# Push to GBQ
ticker_df.to_gbq(destination_table=f'{project_id}.{dataset_id}.ticker_table', project_id=project_id, if_exists='replace')